In [223]:
import pandas as pd
import tensorflow as tf
import keras as ks
import numpy as np
from sklearn.metrics import recall_score, precision_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
import datetime
import time
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, Flatten, Activation, MaxPooling2D
from scripts.model_functions import create_model
import matplotlib.pylab as plt
import seaborn as sns


In [224]:
# Create new variables to be used in Keras and the CNN

# number of items to use for training
BATCH_SIZE = 400 

# Number of identifying classes 
#   WE have two, Bloom and no bloom 1/0
NUM_CLASSES = 2 

# number of times to repeat process
EPOCHS = 100

In [227]:

# Load the data
df_train = pd.read_csv('../../data/cleaned/site1_vineyard.csv')
df_test = pd.read_csv('../../data/cleaned/site2_bird.csv')


In [228]:
target = df_train['BGA-Phycocyanin RFU'].apply(lambda x : x/0.2334)
df_train = df_train.drop(columns=['Chlorophyll (ug/L)', 'Chlorophyll RFU'])
df_train['BGA (ug/L)'] = target
df_train.head(5)


target = df_test['BGA-Phycocyanin RFU'].apply(lambda x : x/0.2334)
df_test = df_test.drop(columns=['Chlorophyll (ug/L)', 'Chlorophyll RFU'])
df_test['BGA (ug/L)'] = target
df_test.head(5)

,Date (mm.dd.yyyy),Time 24hr,Temp C,Sp Cond (uS/cm),pH (mV),pH,Turbidity (NTU),ODOSat%,ODO (mg/L),BGA-Phycocyanin RFU,BGA (ug/L)
0,5/5/2017,0:00,15.37,2184,-100.0,8.41,10.70,92.2,9.16,0.1,0.428449
1,5/5/2017,0:15,15.45,2139,-101.0,8.43,9.92,93.3,9.25,0.1,0.428449
2,5/5/2017,0:30,15.49,2057,-102.3,8.45,8.90,94.8,9.40,0.2,0.856898
3,5/5/2017,0:45,15.67,1978,-102.6,8.45,8.62,96.0,9.49,0.2,0.856898
4,5/5/2017,1:00,15.34,2136,-100.2,8.41,9.88,92.7,9.22,0.1,0.428449


In [229]:

timestamp = df_train['Date (mm.dd.yyyy)'] + ' '+ df_train['Time 24hr']
timestamp = pd.to_datetime(timestamp)
df_train['Timestamp'] = timestamp

timestamp = df_test['Date (mm.dd.yyyy)'] + ' '+ df_test['Time 24hr']
timestamp = pd.to_datetime(timestamp)
df_test['Timestamp'] = timestamp


In [230]:
# dont need data and time now that we have Timestamp. Lets remove them

df_train = df_train.drop(columns=['Date (mm.dd.yyyy)', 'Time 24hr'])
df_test = df_test.drop(columns=['Date (mm.dd.yyyy)', 'Time 24hr'])


In [231]:
train_target = df_train['BGA (ug/L)'].apply(lambda x: 1 if x > 20 else 0)
df_train['Bloom'] = train_target

test_target = df_test['BGA (ug/L)'].apply(lambda x: 1 if x > 20 else 0)
df_test['Bloom'] = test_target


In [232]:
# lets try to normalize this now....
dataset_columns = ['Temp C','Sp Cond (uS/cm)', 'pH (mV)','pH', 'Turbidity (NTU)', 'ODOSat%','ODO (mg/L)', 'Bloom']
scaler = MinMaxScaler()
ds_scaled = scaler.fit_transform(df_train[dataset_columns])
df_train = pd.DataFrame(ds_scaled,columns=dataset_columns)

ds_scaled = scaler.fit_transform(df_test[dataset_columns])
df_test = pd.DataFrame(ds_scaled,columns=dataset_columns)



C:\Users\Public\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\Public\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


# Functions to take a moving window of the data of 10 time stamps

In [233]:
'''
determines the window size for the daata set
@param dataset - The dataset to get windows for
@param window_size - the size of the window  
@param shift - the amout to shift the window
'''
def windows(dataset, window_size, shift):
    start = 0
    while start+window_size < dataset.shape[0]: 
        yield (int(start), int(start+window_size))
        # shift the window five blocks of time
        start += shift
        if start % 300 == 0:
            print('Window Segmentation {0:.2f}% done'.format(((start+window_size) / dataset.shape[0]) * 100 ))


'''

Segments the dataset based on the parameters that are passed in.
@param dataset - the dataset to segment into window
@param columns - the array of columns from the dataset to be looked at
@param window_size - the size of the window you would like to be looked at. Defualt is 10

'''
def segment_dataset(dataset, columns, target, window_size=10):    
    print('WINDOW SIZE',window_size)
    print('NUMBER OF COULUMNS',len(columns))
    segments = np.empty((0, window_size, len(columns)))
    labels = np.empty((0))
    count = 0
    for (start, end) in windows(dataset, window_size, 1):
        count+=1
        values = dataset[columns][start:end]
        if(values.shape[0] == window_size):
            segments = np.vstack([segments, np.stack([values])])
            # Takes the larger of the two variables if there are more than one. 
            # This makes it more likly to predict a bloom. Can be changed to iloc[0] to
            # be less likly to predict a bloom (more 0s in the label array)
            
            labels = np.append(labels, dataset[target][start:end].mode().iloc[-1])
        else:
            print("No more Windows available... Exiting")
            break
    return (segments, labels)

In [234]:
feature_columns = dataset_columns[:-1]
(x_train, y_train) = segment_dataset(df_train, feature_columns, 'Bloom', 9)
(x_test, y_test) = segment_dataset(df_test, feature_columns, 'Bloom', 9)

print('done')

WINDOW SIZE 9
NUMBER OF COULUMNS 7
Window Segmentation 1.63% done
Window Segmentation 3.21% done
Window Segmentation 4.80% done
Window Segmentation 6.38% done
Window Segmentation 7.96% done
Window Segmentation 9.55% done
Window Segmentation 11.13% done
Window Segmentation 12.71% done
Window Segmentation 14.30% done
Window Segmentation 15.88% done
Window Segmentation 17.46% done
Window Segmentation 19.05% done
Window Segmentation 20.63% done
Window Segmentation 22.21% done
Window Segmentation 23.80% done
Window Segmentation 25.38% done
Window Segmentation 26.96% done
Window Segmentation 28.55% done
Window Segmentation 30.13% done
Window Segmentation 31.71% done
Window Segmentation 33.30% done
Window Segmentation 34.88% done
Window Segmentation 36.46% done
Window Segmentation 38.05% done
Window Segmentation 39.63% done
Window Segmentation 41.21% done
Window Segmentation 42.80% done
Window Segmentation 44.38% done
Window Segmentation 45.97% done
Window Segmentation 47.55% done
Window Segm

In [235]:
print(x_train.shape)
print(x_test.shape)


(18938, 9, 7)
(17086, 9, 7)


In [236]:
print(y_train.shape)
print(y_test.shape)

(18938,)
(17086,)


# Shaping the data to be used in the model.

In [237]:
x_train = x_train.reshape(len(x_train),9,7,1)
x_test = x_test.reshape(len(x_test),9,7,1)

In [240]:
y_train = y_train.reshape(y_train.shape[0],1)
y_test = y_test.reshape(y_test.shape[0],1)

# Breaking apart training and test data

In [241]:
print("x_train shape:",x_train.shape)
print("x_test shape:",x_test.shape)
print("y_train shape:",y_train.shape)
print("y_test shape:",y_test.shape)

x_train shape: (18938, 9, 7, 1)
x_test shape: (17086, 9, 7, 1)
y_train shape: (18938, 1)
y_test shape: (17086, 1)


In [242]:
y_train_mod = ks.utils.to_categorical(y_train, NUM_CLASSES)
y_test_mod = ks.utils.to_categorical(y_test, NUM_CLASSES)
input_shape = (9,7,1)


In [243]:

# Gets the precision of the different metrics
def create_class_predictions(pred):
    retval = np.array([])
    for row in pred:
        max_value = (-1,-1)
        for index, value in enumerate(row):
            if value > max_value[1]:
                max_value = (index, value)
        retval = np.append(retval, max_value[0])
    return retval


def create_layers(num_layers):
    layers = [Flatten(), Dropout(0.2), Dense(NUM_CLASSES, activation='softmax', input_dim=2)]
    for i in range(0, num_layers):
        layers.insert(0, Conv2D(44, 7, input_shape=input_shape, activation='relu', padding='same'))
    return layers

# Come on, let's create the model already!

In [244]:
# From the layer/node investigation
# it was found to be the following layers to be the best
# Conv2D 44,4
# Conv2D 44, 7
# Conv2D 44, 4
# Conv2D 44 7
# Flatten()
# Dropout(0.2)
# Dense(44)
# Dense(2)

# let's train a model to see if we get similar results with that
model = Sequential()
model.add(Conv2D(44, 7, input_shape=input_shape, activation='relu', padding='same'))
model.add(Conv2D(44, 4, activation='relu', padding='same'))
model.add(Conv2D(44, 4, activation='relu', padding='same'))
model.add(Conv2D(44, 7, activation='relu', padding='same'))
model.add(Flatten())
model.add(Dropout(0.2)) 
model.add(Dense(44))
model.add(Dense(NUM_CLASSES, activation='softmax'))
model.compile(loss=ks.losses.categorical_crossentropy,
        optimizer=ks.optimizers.Adam(lr=0.0001),
        metrics=['accuracy'])
model.fit(x=x_train, y=y_train_mod, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1)
score = model.evaluate(x_train, y_train_mod, verbose=1)


Epoch 1/100
18938/18938 [==============================] - 5s 267us/step - loss: 0.1827 - acc: 0.9914
Epoch 2/100
18938/18938 [==============================] - 1s 41us/step - loss: 0.0299 - acc: 0.9961
Epoch 3/100
18938/18938 [==============================] - 1s 40us/step - loss: 0.0260 - acc: 0.9961
Epoch 4/100
18938/18938 [==============================] - 1s 41us/step - loss: 0.0213 - acc: 0.9961
Epoch 5/100
18938/18938 [==============================] - 1s 41us/step - loss: 0.0171 - acc: 0.9961
Epoch 6/100
18938/18938 [==============================] - 1s 40us/step - loss: 0.0166 - acc: 0.9959
Epoch 7/100
18938/18938 [==============================] - 1s 40us/step - loss: 0.0154 - acc: 0.9961
Epoch 8/100
18938/18938 [==============================] - 1s 41us/step - loss: 0.0158 - acc: 0.9960
Epoch 9/100
18938/18938 [==============================] - 1s 41us/step - loss: 0.0146 - acc: 0.9961
Epoch 10/100
18938/18938 [==============================] - 1s 41us/step - loss: 0.0142 - 

In [245]:
# Evaluate the model
predictions = model.predict(x_test)
predict = create_class_predictions(predictions)
y = y_test.reshape(-1,)
recall = recall_score(y, predict)
precision = precision_score(y, predict)
print("Recall:",recall)
print("Precision:", precision)
print(confusion_matrix(y, predict))

Recall: 0.0
Precision: 0.0
[[17073     0]
 [   13     0]]


C:\Users\Public\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


## Save the model for deployment

In [ ]:
# ignoring dropout for deployment
K.set_learning_phase(0)
 
# Set a file path to save the model in.
model_name = "cnn_model"
model_version = "1"
tf_path = "./../../saved_models/{}/{}".format(model_name, model_version)
 
# Get the session from the Keras back-end to save the model in TF format.
with K.get_session() as sess:
    tf.saved_model.simple_save(sess, tf_path, inputs={'input': model.input}, outputs={t.name: t for t in model.outputs})
